In [1]:
import numpy as np
import pandas as pd

from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

import pickle

# Import dataset

In [2]:
data = pd.read_csv("../DataFormating/final.csv")

In [3]:
data.head()

,Away Team Goals,Away Team Name,Home Team Goals,Home Team Name,Year,Home Avg Goals,Away Avg Goals,Home FIFA Points,Away FIFA Points
0,1.0,Mexico,4.0,France,1930.0,0.0,0.0,1166.0,1008.0
1,1.0,Spain,0.0,Portugal,1930.0,0.0,0.0,1306.0,1162.0
2,3.0,Poland,0.0,Sweden,1930.0,0.0,0.0,889.0,1118.0
3,2.0,Sweden,2.0,Belgium,1930.0,0.0,0.0,1346.0,889.0
4,3.0,Germany,6.0,Denmark,1930.0,0.0,0.0,1054.0,1533.0


# Set `X` and `y`

In [4]:
data.columns

Index(['Away Team Goals', 'Away Team Name', 'Home Team Goals',
       'Home Team Name', 'Year', 'Home Avg Goals', 'Away Avg Goals',
       'Home FIFA Points', 'Away FIFA Points'],
      dtype='object')

In [5]:
X = data.drop(["Away Team Goals", "Home Team Goals"], axis=1)

y = []
for i in range(len(data)):
    home_team_goals = data.iloc[i]["Home Team Goals"]
    away_team_goals = data.iloc[i]["Away Team Goals"]
    
    if home_team_goals > away_team_goals:
        y.append(1)
    elif home_team_goals < away_team_goals:
        y.append(2)
    else:
        y.append(0)

In [6]:
# Test
assert len(X) == len(y)

### Encode textual features from the `X` dataset

In [7]:
word_cup_teams = [
    "Egypt",
    "Morocco",
    "Nigeria",
    "Senegal",
    "Tunisia",
    "Australia",
    "Iran",
    "Japan",
    "Korea DPR",
    "Saudi Arabia",
    "Belgium",
    "Croatia",
    "Denmark",
    "England",
    "France",
    "Germany",
    "Iceland",
    "Poland",
    "Portugal",
    "Russia",
    "Serbia",
    "Spain",
    "Sweden",
    "Switzerland",
    "Costa Rica",
    "Mexico",
    "Panama",
    "Argentina",
    "Brazil",
    "Colombia",
    "Peru",
    "Uruguay"
]

team_names = list(data["Home Team Name"].unique()) + list(data["Away Team Name"].unique()) + word_cup_teams

In [8]:
team_name_encoder = LabelEncoder().fit(team_names)

In [9]:
X["Home Team Name"] = team_name_encoder.transform(X["Home Team Name"])
X["Away Team Name"] = team_name_encoder.transform(X["Away Team Name"])

### Feature Selection

In [10]:
feature_names = [
    "Away Team Name",
    "Home Team Name",
    "Avg Goals",
    "FIFA Points"
]

COLUMNS = []
for column_name in X.columns:
    for feature_name in feature_names:
        if feature_name in column_name:
            COLUMNS.append(column_name)
            break

X = X[COLUMNS]

In [11]:
COLUMNS

['Away Team Name',
 'Home Team Name',
 'Home Avg Goals',
 'Away Avg Goals',
 'Home FIFA Points',
 'Away FIFA Points']

# Traning Session

In [12]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3951 entries, 0 to 3950
Data columns (total 6 columns):
Away Team Name      3951 non-null int64
Home Team Name      3951 non-null int64
Home Avg Goals      3951 non-null float64
Away Avg Goals      3951 non-null float64
Home FIFA Points    3951 non-null float64
Away FIFA Points    3951 non-null float64
dtypes: float64(4), int64(2)
memory usage: 185.3 KB


In [13]:
X.describe()

,Away Team Name,Home Team Name,Home Avg Goals,Away Avg Goals,Home FIFA Points,Away FIFA Points
count,3951.000000,3951.000000,3951.000000,3951.000000,3951.000000,3951.000000
mean,15.936978,14.580359,1.448520,1.024867,1031.470008,1010.762086
std,10.283246,10.158643,1.245904,0.960996,287.679806,276.569105
min,0.000000,0.000000,0.000000,0.000000,445.000000,445.000000
25%,7.000000,5.000000,0.500000,0.000000,889.000000,858.000000
50%,16.000000,13.000000,1.333333,1.000000,1040.000000,1040.000000
75%,25.000000,23.000000,2.080000,1.500000,1254.000000,1179.000000
max,32.000000,32.000000,9.000000,10.000000,1533.000000,1533.000000


In [14]:
xgb_model = XGBClassifier(
    n_estimators=500,
    max_depth=3,
    learning_rate=0.03,
    booster="gbtree",
    n_jobs=-1
)

In [15]:
xgb_model.fit(X, y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.03, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=500,
       n_jobs=-1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

# Save model and encoders

In [16]:
with open("xgb_model.b", "wb") as f:
    pickle.dump(xgb_model, f)
    
with open("team_name_encoder.b", "wb") as f:
    pickle.dump(team_name_encoder, f)